In [1]:
import numpy as np
from scipy.signal import convolve
import mat73
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.io as sio
import time

In [2]:
class FDK(object):
    
    """The FDK Method of 3D Image Reconstruction"""
    
    def __init__(self, fname):
        stime = time.time()
        raw_data = np.fromfile(fname, np.float32)
        # read / write the elements using Fortran-like / MATLAB-like index order
        self.data = raw_data.reshape(256, 256, 360, order='F')  # reshape the data as (b, a, beta)
        self.data = raw_data.reshape(360, 256, 256) # reshape the data as (beta, a, b)
        self.kn = np.linspace(-255, 255, 511)    # convolution kernel
        self.l = 20 # virtual detector length
        self.n = 256    # detector number
        self.T = self.l / self.n   # per detector length
        self.a = np.linspace((self.l - self.T) / 2, (-self.l + self.T) / 2, self.n)[np.newaxis, :, np.newaxis] # a array
        self.b = np.linspace((self.l - self.T) / 2, (-self.l + self.T) / 2, self.n)[np.newaxis, np.newaxis, :] # b array
        self.R = 40 # the distance between source and rotational center
        #print(self.R / np.sqrt(self.R ** 2 + self.a ** 2 + self.b ** 2))
        #print(np.shape(self.R / np.sqrt(self.R ** 2 + self.a ** 2 + self.b ** 2)))
        self.x_array = np.linspace((-self.l + self.T) / 2, (self.l - self.T) / 2, self.n)[:, np.newaxis, np.newaxis]
        self.y_array = np.linspace((-self.l + self.T) / 2, (self.l - self.T) / 2, self.n)[np.newaxis, :, np.newaxis]
        self.z_array = np.linspace((-self.l + self.T) / 2, (self.l - self.T) / 2, self.n)[np.newaxis, np.newaxis, :]
        self.beta_array = np.arange(0, 2 * np.pi, np.pi / 180)
        self.rf = self.ramp_filter()
        self.pw_data = self.pre_weighting()
        self.convolved_data = self.convolve3D()
        #print(np.shape(self.convolved_data))
        #print(self.convolved_data[180,128])
        self.f_fdk = np.zeros((256, 256, 256))
        for beta in self.beta_array:
            U = self.R + self.x_array * np.cos(beta) + self.y_array * np.sin(beta)
            a = self.R / U * (-self.x_array * np.sin(beta) + self.y_array * np.cos(beta))
            b = self.R / U * self.z_array
            print(a)
            print("-"*50)
            print(b)
            print("*"*50)
            a_around = (np.around((a - self.T / 2) / self.T) * self.T + self.T / 2)[:, ::-1, :]
            b_around = (np.around((b - self.T / 2) / self.T) * self.T + self.T / 2)[:, :, ::-1]
            a_around[a_around > ((self.l - self.T) / 2)] = (self.l - self.T) / 2
            a_around[a_around < ((-self.l + self.T) / 2)] = (-self.l + self.T) / 2
            b_around[b_around > ((self.l - self.T) / 2)] = (self.l - self.T) / 2
            b_around[b_around < ((-self.l + self.T) / 2)] = (-self.l + self.T) / 2
            print(a_around)
            print("-"*50)
            print(b_around)
            print("*"*50)
            a_index = ((a_around + (self.l - self.T) / 2) / self.T).astype(int)
            print(a_index)
            print("-"*50)
            b_index = ((b_around + (self.l - self.T) / 2) / self.T).astype(int)
            print(b_index)
            print("*"*50)
            # np.savetxt("a_index.txt", a_index)
            print(int(beta*180/np.pi))
            f = self.R**2 / U**2 * self.convolved_data[int(beta*180/np.pi), a_index, b_index]
            self.f_fdk += f
        mdic = {"fdk_shepplogan": self.f_fdk}
        sio.savemat("./data/fdk_shepplogan.mat", mdic)
        etime = time.time()
        print(np.around(etime - stime))
    def ramp_filter(self):
        return -2 / ((np.pi * self.T) ** 2 * (4 * self.kn ** 2 - 1))[np.newaxis, :, np.newaxis]

    def pre_weighting(self):
        return self.R * self.data / np.sqrt(self.R ** 2 + self.a ** 2 + self.b ** 2)

    def convolve3D(self):
        return convolve(self.pw_data, self.rf, mode='same', method='auto')

In [3]:
class Analysis(object):
    def __init__(self):
        original = mat73.loadmat('data/Shepplogan.mat')['Shepplogan']
        fdk = sio.loadmat('data/fdk_shepplogan.mat')["fdk_shepplogan"]
        print(original)
        print(fdk)

In [4]:
if __name__ == "__main__":
    ct = FDK("./data/Circular CBCT_flat_panel_detector.prj")
    #a = Analysis()

[[[-13.26397919]
  [-13.15994798]
  [-13.05591678]
  ...
  [ 13.05591678]
  [ 13.15994798]
  [ 13.26397919]]

 [[-13.22957198]
  [-13.12581064]
  [-13.02204929]
  ...
  [ 13.02204929]
  [ 13.12581064]
  [ 13.22957198]]

 [[-13.19534282]
  [-13.09184994]
  [-12.98835705]
  ...
  [ 12.98835705]
  [ 13.09184994]
  [ 13.19534282]]

 ...

 [[ -8.        ]
  [ -7.9372549 ]
  [ -7.8745098 ]
  ...
  [  7.8745098 ]
  [  7.9372549 ]
  [  8.        ]]

 [[ -7.98747063]
  [ -7.92482381]
  [ -7.86217698]
  ...
  [  7.86217698]
  [  7.92482381]
  [  7.98747063]]

 [[ -7.97498045]
  [ -7.91243159]
  [ -7.84988272]
  ...
  [  7.84988272]
  [  7.91243159]
  [  7.97498045]]]
--------------------------------------------------
[[[-13.26397919 -13.15994798 -13.05591678 ...  13.05591678  13.15994798
    13.26397919]
  [-13.26397919 -13.15994798 -13.05591678 ...  13.05591678  13.15994798
    13.26397919]
  [-13.26397919 -13.15994798 -13.05591678 ...  13.05591678  13.15994798
    13.26397919]
  ...
  [-13.263

UnboundLocalError: local variable 'b_index' referenced before assignment